<a href="https://colab.research.google.com/github/adidror005/youtube-videos/blob/main/LLAMA_3_Fine_Tuning_for_Sequence_Classification_Actual_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLAMA3 Fine-tuning for text classification using QLORA


### Requirements:
* A GPU with enough memory!

### Installs
* They suggest using latest version of transformers
* Must restart after install because the accelerate package used in the hugging face trainer requires it.

In [1]:
# Install Pytorch
%pip install "torch==2.2.2" tensorboard

# Install Hugging Face libraries
%pip install  --upgrade "transformers==4.40.0" "datasets==2.18.0" "accelerate==0.29.3" "evaluate==0.4.1" "bitsandbytes==0.43.1" "huggingface_hub==0.22.2" "trl==0.8.6" "peft==0.10.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

### Login to huggingface hub to put your LLama token so we can access Llama 3 8B Param Pre-trained Model

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in c

#### Imports

In [68]:
import os
import random
import functools
import csv
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import evaluate

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, classification_report, balanced_accuracy_score, accuracy_score

from datasets import Dataset, DatasetDict
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)

#### Load DataFrame

In [69]:
df = pd.read_excel("/content/ft_data.xlsx")
df.head()

,intent,query
0,Card: disable,"Need to block my card ASAP, think it's been co..."
1,Card: disable,"Freeze my card, pretty sure I left it at the b..."
2,Card: disable,Can you put a hold on my card? I can't find it...
3,Card: disable,"Ugh, lost my wallet. Disable my card before so..."
4,Card: disable,"Hey, I think my card's been nicked! Lock it do..."


In [70]:
# Add also a numeric 0,1,2 version of label since we will need it later for fine tuning. We can save it in 'target'
df['intent']=df['intent'].astype('category')
df['target']=df['intent'].cat.codes

df

,intent,query,target
0,Card: disable,"Need to block my card ASAP, think it's been co...",2
1,Card: disable,"Freeze my card, pretty sure I left it at the b...",2
2,Card: disable,Can you put a hold on my card? I can't find it...,2
3,Card: disable,"Ugh, lost my wallet. Disable my card before so...",2
4,Card: disable,"Hey, I think my card's been nicked! Lock it do...",2
...,...,...,...
875,User Account: change password post login,Show me steps to change my user account password.,34
876,User Account: change password post login,"I suspect my password is compromised, how do I...",34
877,User Account: change password post login,What's the procedure to update my login passwo...,34
878,User Account: change password post login,"I'd like to set a new password for my account,...",34


In [71]:
df['intent'].cat.categories

Index(['Card: add new', 'Card: cancel close', 'Card: disable', 'Card: enable',
       'Card: get info status', 'Card: get shipping status where is',
       'Card: replace or upgrade', 'Card: report not received',
       'Card: report stolen or lost', 'Card: reset PIN',
       'Card: temporary limit increase', 'Document: upload',
       'Global: get balance', 'Global: get routing number direct deposit info',
       'Notifications: manage', 'Notifications: sign up for',
       'Overdraft: opt out', 'Refer a Friend: get info', 'Rewards: opt in',
       'Rewards: opt out', 'Rewards: view offers',
       'Savings Account: get info view program',
       'Spend Account: consent to direct deposit', 'Spend Account: find ATMs',
       'Spend Account: get cash withdrawal and reload locations',
       'Spend Account: transfer funds', 'Spend Account: transfer funds checks',
       'Spend Account: transfer funds external bank',
       'Spending Tracker: get info', 'Statement: get', 'Transaction: his

In [72]:
category_map = {code: category for code, category in enumerate(df['intent'].cat.categories)}
category_map

{0: 'Card: add new',
 1: 'Card: cancel close',
 2: 'Card: disable',
 3: 'Card: enable',
 4: 'Card: get info status',
 5: 'Card: get shipping status where is',
 6: 'Card: replace or upgrade',
 7: 'Card: report not received',
 8: 'Card: report stolen or lost',
 9: 'Card: reset PIN',
 10: 'Card: temporary limit increase',
 11: 'Document: upload',
 12: 'Global: get balance',
 13: 'Global: get routing number direct deposit info',
 14: 'Notifications: manage',
 15: 'Notifications: sign up for',
 16: 'Overdraft: opt out',
 17: 'Refer a Friend: get info',
 18: 'Rewards: opt in',
 19: 'Rewards: opt out',
 20: 'Rewards: view offers',
 21: 'Savings Account: get info view program',
 22: 'Spend Account: consent to direct deposit',
 23: 'Spend Account: find ATMs',
 24: 'Spend Account: get cash withdrawal and reload locations',
 25: 'Spend Account: transfer funds',
 26: 'Spend Account: transfer funds checks',
 27: 'Spend Account: transfer funds external bank',
 28: 'Spending Tracker: get info',
 29: 

In [74]:
# category_map

In [75]:
df.pop('intent')
df.head()

,query,target
0,"Need to block my card ASAP, think it's been co...",2
1,"Freeze my card, pretty sure I left it at the b...",2
2,Can you put a hold on my card? I can't find it...,2
3,"Ugh, lost my wallet. Disable my card before so...",2
4,"Hey, I think my card's been nicked! Lock it do...",2


### Convert from Pandas DataFrame to Hugging Face Dataset
* train/val/test split (80/10/10)
* Shuffle the training set.
* We put the components train,val,test into a DatasetDict so we can access them later with HF trainer.
* Later we will add a tokenized dataset

In [76]:
# Shuffle the DataFrame
df_shuffled = df.sample(frac=1, random_state=42)

In [77]:
# Define the sizes for train, val, and test sets
train_size = int(df_shuffled.shape[0] * 0.8)
val_size = int(df_shuffled.shape[0] * 0.1)

# Split the shuffled DataFrame into train, val, and test sets
df_train = df_shuffled.iloc[:train_size]
df_val = df_shuffled.iloc[train_size:train_size + val_size]
df_test = df_shuffled.iloc[train_size + val_size:]

print(df_train.shape, df_val.shape, df_test.shape)

(704, 2) (88, 2) (88, 2)


In [78]:
# Converting pandas DataFrames into Hugging Face Dataset objects:
dataset_train = Dataset.from_pandas(df_train.reset_index(drop=True))
dataset_val = Dataset.from_pandas(df_val.reset_index(drop=True))
dataset_test = Dataset.from_pandas(df_test.reset_index(drop=True))

In [79]:
# Combine them into a single DatasetDict
dataset = DatasetDict({
    'train': dataset_train,
    'val': dataset_val,
    'test': dataset_test
})
dataset

DatasetDict({
    train: Dataset({
        features: ['query', 'target'],
        num_rows: 704
    })
    val: Dataset({
        features: ['query', 'target'],
        num_rows: 88
    })
    test: Dataset({
        features: ['query', 'target'],
        num_rows: 88
    })
})

In [80]:
dataset['train'][:5]

{'query': ['Show me the list of rewards I can redeem with my points.',
  'Guide me through check transfer from my spend account, please?',
  'How do I turn off transaction alerts on my account?',
  "I'd like to consolidate my banking by adding another account. How do I do that?",
  "What's the procedure to modify the address linked to my account?"],
 'target': [20, 26, 14, 37, 33]}

## Load LLama model with 4 bit quantization as specified in bits and bytes and prepare model for peft training

### Model Name

In [81]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

#### Quantization Config (for QLORA)

In [82]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, # enable 4-bit quantization
    bnb_4bit_quant_type = 'nf4', # information theoretically optimal dtype for normally distributed weights
    bnb_4bit_use_double_quant = True, # quantize quantized weights //insert xzibit meme
    bnb_4bit_compute_dtype = torch.bfloat16 # optimized fp format for ML
)

#### Lora Config

In [83]:
lora_config = LoraConfig(
    r = 16, # the dimension of the low-rank matrices
    lora_alpha = 8, # scaling factor for LoRA activations vs pre-trained weight activations
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout = 0.05, # dropout probability of the LoRA layers
    bias = 'none', # wether to train bias weights, set to 'none' for attention layers
    task_type = 'SEQ_CLS'
)

#### Load model
* AutomodelForSequenceClassification
* Num Labels is # of classes


In [84]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    num_labels=44
)

model

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


* prepare_model_for_kbit_training() function to preprocess the quantized model for training.

In [85]:
model = prepare_model_for_kbit_training(model)
model

LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


* get_peft_model prepares a model for training with a PEFT method such as LoRA by wrapping the base model and PEFT configuration with get_peft_model

In [86]:
model = get_peft_model(model, lora_config)
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): LlamaForSequenceClassification(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
        

### Load the tokenizer

#### Since LLAMA3 pre-training doesn't have EOS token
* Set the pad_token_id to eos_token_id
* Set pad token ot eos_token

In [88]:
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### Update some model configs

In [89]:
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False # use_cache = False or it might crash
model.config.pretraining_tp = 1

# Trainer Components
* model
* tokenizer
* training arguments
* train dataset
* eval dataset
* Data Collater
* Compute Metrics
* class_weights: In our case since we are using a custom trainer so we can use a weighted loss we will subclass trainer and define the custom loss.

#### Create LLAMA tokenized dataset which will house our train/val parts during the training process but after applying tokenization

In [90]:
MAX_LEN = 512

def llama_preprocessing_function(examples):
    return tokenizer(examples['query'], truncation=True, max_length=MAX_LEN)

tokenized_datasets = dataset.map(llama_preprocessing_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("target", "label")
tokenized_datasets.set_format("torch")

Map:   0%|          | 0/704 [00:00<?, ? examples/s]

Map:   0%|          | 0/88 [00:00<?, ? examples/s]

Map:   0%|          | 0/88 [00:00<?, ? examples/s]

## Data Collator

1. **Padding:** Uniformly pads sequences to the length of the longest sequence using a special token, allowing simultaneous batch processing.
2. **Batching:** Groups individual data points into batches for efficient processing.
3. **Handling Special Tokens:** Adds necessary special tokens to sequences.
4. **Converting to Tensor:** Transforms data into tensors, the required format for machine learning frameworks.

### `DataCollatorWithPadding`

The `DataCollatorWithPadding` specifically manages padding, using a tokenizer to ensure that all sequences are padded to the same length for consistent model input.

- **Syntax:** `collate_fn = DataCollatorWithPadding(tokenizer=tokenizer)`
- **Purpose:** Automatically pads text data to the longest sequence in a batch, crucial for models like BERT or GPT.
- **Tokenizer:** Uses the provided `tokenizer` for sequence processing, respecting model-specific vocabulary and formatting rules.

In [91]:
collate_fn = DataCollatorWithPadding(tokenizer=tokenizer)

### define which metrics to compute for evaluation

In [92]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'balanced_accuracy' : balanced_accuracy_score(predictions, labels),'accuracy':accuracy_score(predictions,labels)}

### Define custom trainer with classweights
* We will have a custom loss function that deals with the class weights and have class weights as additional argument in constructor

In [93]:
class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        # Ensure label_weights is a tensor
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)
        else:
            self.class_weights = None

    def compute_loss(self, model, inputs, return_outputs=False):
        # Extract labels and convert them to long type for cross_entropy
        labels = inputs.pop("labels").long()

        # Forward pass
        outputs = model(**inputs)

        # Extract logits assuming they are directly outputted by the model
        logits = outputs.get('logits')

        # Compute custom loss with class weights for imbalanced data handling
        if self.class_weights is not None:
            loss = F.cross_entropy(logits, labels, weight=self.class_weights)
        else:
            loss = F.cross_entropy(logits, labels)

        return (loss, outputs) if return_outputs else loss


#### define training args

In [94]:
training_args = TrainingArguments(
    output_dir = 'intent_classification',
    learning_rate = 1e-4,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 2,
    weight_decay = 0.01,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True
)

#### Define custom trainer

In [97]:
trainer = CustomTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['val'],
    tokenizer = tokenizer,
    data_collator = collate_fn,
    compute_metrics = compute_metrics,
    # class_weights=class_weights,
)

### Run trainer!

In [98]:
train_result = trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Balanced Accuracy,Accuracy
1,No log,2.106108,0.453210,0.420455
2,No log,0.741275,0.833333,0.818182


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [99]:
df_test

,query,target
161,There's a weird transaction on my statement. H...,31
555,Provide me with a map of reload locations for ...,24
729,"I'm planning a one-time purchase, can my card ...",10
401,"Show me the spending tracker page, please?",28
702,"I don't need this card anymore, can you perman...",1
...,...,...
106,Hook me up with that routing number for wiring...,13
270,How do I transfer money to an external bank ac...,27
860,"Yo, how do I swap my password after I've logge...",34
435,"I want to opt out of the overdraft service, wh...",16


#### Let's check the results

In [100]:
def make_predictions(model,df_test):

  # Convert Queries to a list
  sentences = list(df_test['query'])

  # Define the batch size
  batch_size = 32

  # Initialize an empty list to store the model outputs
  all_outputs = []

  # Process the sentences in batches
  for i in range(0, len(sentences), batch_size):
      # Get the batch of sentences
      batch_sentences = sentences[i:i + batch_size]

      # Tokenize the batch
      inputs = tokenizer(batch_sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)

      # Move tensors to the device where the model is (e.g., GPU or CPU)
      inputs = {k: v.to('cuda' if torch.cuda.is_available() else 'cpu') for k, v in inputs.items()}

      # Perform inference and store the logits
      with torch.no_grad():
          outputs = model(**inputs)
          all_outputs.append(outputs['logits'])
  final_outputs = torch.cat(all_outputs, dim=0)
  df_test['predictions']=final_outputs.argmax(axis=1).cpu().numpy()
  df_test['predictions']=df_test['predictions'].apply(lambda l:list(category_map.keys())[list(category_map.values()).index(l)])


make_predictions(model,df_test)

<ipython-input-100-189c8e9b0286>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predictions']=final_outputs.argmax(axis=1).cpu().numpy()


ValueError: 31 is not in list

In [101]:
df_test

,query,target,predictions
161,There's a weird transaction on my statement. H...,31,31
555,Provide me with a map of reload locations for ...,24,24
729,"I'm planning a one-time purchase, can my card ...",10,10
401,"Show me the spending tracker page, please?",28,28
702,"I don't need this card anymore, can you perman...",1,1
...,...,...,...
106,Hook me up with that routing number for wiring...,13,13
270,How do I transfer money to an external bank ac...,27,27
860,"Yo, how do I swap my password after I've logge...",34,34
435,"I want to opt out of the overdraft service, wh...",16,16


In [62]:
# df_test['predictions_target']=df_test['predictions'].apply(lambda l:list(category_map.keys())[list(category_map.values()).index(l)])

In [102]:

def get_performance_metrics(df_test):
  y_test = df_test.target
  y_pred = df_test.predictions

  print("Confusion Matrix:")
  print(confusion_matrix(y_test, y_pred))

  print("\nClassification Report:")
  print(classification_report(y_test, y_pred))

  print("Balanced Accuracy Score:", balanced_accuracy_score(y_test, y_pred))
  print("Accuracy Score:", accuracy_score(y_test, y_pred))

In [103]:
get_performance_metrics(df_test)

Confusion Matrix:
[[1 0 0 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 ...
 [0 0 0 ... 3 0 0]
 [0 0 0 ... 0 2 0]
 [0 1 0 ... 0 0 5]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       0.67      1.00      0.80         2
           2       0.67      1.00      0.80         2
           3       1.00      0.67      0.80         3
           5       1.00      1.00      1.00         1
           7       0.50      0.50      0.50         2
           8       0.50      1.00      0.67         1
           9       1.00      1.00      1.00         3
          10       1.00      1.00      1.00         1
          11       0.75      1.00      0.86         3
          12       1.00      0.75      0.86         4
          13       1.00      1.00      1.00         3
          14       0.00      0.00      0.00         1
          15       0.50      1.00      0.67         2
          16       1.00    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

### Saving the model trainer state and model adapters

In [104]:
metrics = train_result.metrics
max_train_samples = len(dataset_train)
metrics["train_samples"] = min(max_train_samples, len(dataset_train))
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

***** train metrics *****
  epoch                    =        2.0
  total_flos               =  1012945GF
  train_loss               =     2.4284
  train_runtime            = 0:11:00.67
  train_samples            =        704
  train_samples_per_second =      2.131
  train_steps_per_second   =      0.266


#### Saving the adapter model
* Note this doesn't save the entire model. It only saves the adapters.

In [105]:
trainer.save_model("saved_model")